In [1]:
#!pip install transformers

In [2]:
import nltk
nltk.download("popular")

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/talos-1/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /home/talos-1/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /home/talos-1/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /home/talos-1/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /home/talos-1/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /home/talos-1/nltk_data...
[nltk_data]    |   Package movie_reviews is already

True

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import cross_val_score
import torch
import transformers as ppb
# import warnings
from nltk.corpus import wordnet
import random
from sklearn.neural_network import MLPClassifier
# import tensorflow as tf
from sklearn.metrics import confusion_matrix
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [4]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

2025-11-20 15:11:27.320039: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-20 15:11:27.460478: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-20 15:11:29.630758: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


### Function Definitions

In [5]:
#Generator Function for Non-Numeric Words
#X is a pandas Series of a sentence
#here gamma is the number of iterations
#returns a series of modified messages (might be modified)


#Function takes a sequence or word and produces a modified version.
# If the dataset has less unique works than half it's set, it randomly resamples.
# Otherwise:
  #It splits the sentence into words.
  #Identifies adjectives using WordNet (lexical db of semantic relations between words).
  #Chooses a word in the sentence (prefers adjectives) and looks up the antonym.
  #If there is an antonym, it replaces the word with one of them, using gamma.
  #Otherwise is leaves the sentence unchanged.

#Note: Lemmatisation works by reducing a word to it's base form.
# Meaning "running", "ran", and "runs" are all lemmatised to "run".

def generatorWords(X,gamma):
  fakeX = []

  if len(X.unique()) < len(X)/2: #if the number of unique values is less than half of the set
    uniqueVals = X.unique()
    fakeX = []
    for _ in range(len(X)):
      fakeX.append(random.choice(uniqueVals))
  else:
    for _, value in X.items(): 
      words = value.split(" ")
      adjs = []
      choosenWord = ""
      pos_all = dict()
      antonyms = []

      #priortize adjectives
      for w in words:
        #print(w)
        pos_l = set()
        for tmp in wordnet.synsets(w): #alias 'wn' not aliased.
            if tmp.name().split('.')[0] == w:
                pos_l.add(tmp.pos())
        pos_all[w] = pos_l

      #in the list of word classes we single out the adjectives
      for a in pos_all:  
        if "a" in pos_all[a]:
          adjs.append(a)
      
      #here we choose which word to flip
      if adjs: 
        choosenWord = adjs[0] #adjs[0] if len(adjs) > 1 else adjs[0]
      else:
        choosenWord = words[0] #words[0] if len(words) > 1 else words[0]

      #find antonyms of the choosen word
      for syn in wordnet.synsets(choosenWord):
        for lm in syn.lemmas():
            if lm.antonyms():
                antonyms.append(lm.antonyms()[0].name())
      if antonyms:
        new = [antonyms[gamma] if x==choosenWord else x for x in words]
      else:
        new = words
        # new = [x for x in words if x == choosenWord] #[x if x == choosenWord else x for x in words]
      fakeX.append(" ".join(new))
  return pd.Series(fakeX)


In [6]:
#Generator Function For Numeric Data Points
#X is a series of numbers
#gamma = 1.4
#intX = list(map(int, X)) #make sure X is an int
def generatorNum(X,gamma):
  xHat = np.mean(X)
  sx = np.std(X)
  randomUpper = (xHat) + (sx * gamma)
  randomLower = (xHat) - (sx * gamma)
  beta = []
  for _ in range(int(len(X))):
      beta.append(random.randrange(int(randomLower),int(randomUpper)))
  beta = list(map(str, beta))
  #print(beta)
  return pd.Series(beta)

In [7]:
#Generator That Points Data Into The Right Generator
def generator(df, gamma):
  mDF = df.copy()
  mDF["temperature"] = generatorNum(df["temperature"],gamma)
  mDF["outlook"] = generatorWords(df["outlook"],0)
  mDF["wind"] = generatorWords(df["wind"],0)
  mDF["humidity"] = generatorNum(df["humidity"],gamma)
  fakeData = addLabel(makeSentence(mDF),1) #column 0 is sentences, column 1 is label
  realData = addLabel(makeSentence(df),0)  #Label 0 means data, label 1 means fake data
  return(pd.concat([fakeData,realData],axis=0),fakeData)

In [8]:
#Merges All DataFrame Columns Into A Single Sentence
#Returns a series of the merged DataFrame columns
def makeSentence(df):
  #df is a data frame
  df = df.copy().map(str)
  seriesA = df[df.columns[0:]].apply(lambda x: ','.join(x.dropna().astype(str)),axis=1)
  return seriesA

In [9]:
#Adds a label columns to series for classification
#Returns dataframe of two columns, the series and the labels
def addLabel(series,label):
  #takes in a pandas series and a label
  #returns a dataframe
  labels = []
  [labels.append(label) for _ in range(len(series))]
  labels = pd.Series(labels)
  return pd.concat([series,labels],axis=1)

In [10]:
#Discriminiator for deteremining whether data is malicious or not
#Returns new gamma, other parameters are for testing and graphing purposes
def discriminator(X, gamma, rho):
  #takes in dataframe in BERT format of sentence and label
  #returns a new gamme to generate data with
  score,indexes,fn,total = BERT(X)
  print("score: " + str(score))
  #if 0.35 < score < 0.55:
  ##  return None
  #else:
  return (score,gamma + rho,indexes,fn,total)

In [11]:
def BERT(df):
  #df is a dataframe containing sentences and labels
  #BERT returns a accuracy and indexes of false negatives

  #tokenize
  tokenized = df[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
  max_len = 0
  for i in tokenized.values:
      if len(i) > max_len:
          max_len = len(i)

  padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values]) #pad
  attention_mask = np.where(padded != 0, 1, 0)                            #apply masks
  input_ids = torch.tensor(padded)  
  attention_mask = torch.tensor(attention_mask)                           #put masks through torch tensor


  #run through bert
  with torch.no_grad():
      last_hidden_states = model(input_ids, attention_mask=attention_mask)
  features = last_hidden_states[0][:,0,:].numpy()
  labels = df[1]
  
  return(regression(features,labels))

In [12]:
#Takes BERT pretrained output and put's it through logistic regression
#Returns accuracy and indexes of false negatives
def regression(features,labels):
  train_features, test_features, train_labels, test_labels = train_test_split(features, labels, random_state=42)
  #lr_clf = LogisticRegression(max_iter=500)
  #lr_clf.fit(train_features, train_labels)
  #return(lr_clf.score(test_features, test_labels))
  NeuralNetwork = MLPClassifier(solver='adam', activation="logistic", random_state=1,max_iter=5000)
  NeuralNetwork.fit(train_features, train_labels)
  y2 = NeuralNetwork.predict(test_features)
  y2 = pd.Series(NeuralNetwork.predict(test_features))
  #print(test_labels)
  i=0
  indexes = []
  for index, value in test_labels.items(): 
    if value == 1:
      if y2[i] == 0:
        indexes.append(index)
    i+=1
  #print(indexes)
  tn, fp, fn, tp = confusion_matrix(test_labels, y2).ravel()
  #fp = confusion_matrix(test_labels, y2)[0][1]
  total = len(test_labels)
  return (NeuralNetwork.score(test_features,test_labels),indexes,fn,total)

In [13]:
def indexToEntry(X,indexes):
  #finds list of indexes in X and returns a dataframe with the entries
  entries = []
  for i in indexes:
    entries.append(X.iloc[i])
  return(pd.DataFrame(entries))

In [14]:
# def logisticRegression(train_features, test_features, train_labels, test_labels):
#   lr_clf = LogisticRegression(max_iter=500)
#   lr_clf.fit(train_features, train_labels)
#   #print(lr_clf.score(test_features, test_labels))
#   y_pred = lr_clf.predict(test_features)
#   score = lr_clf.score(test_features, test_labels)
#   tn, fp, fn, tp = confusion_matrix(test_labels, y_pred).ravel()
#   return(fn,score)

In [15]:
def logisticRegression(train_features, test_features, train_labels, test_labels):
    from sklearn.pipeline import make_pipeline
    from sklearn.preprocessing import StandardScaler
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import confusion_matrix

    clf = make_pipeline(
        StandardScaler(),
        LogisticRegression(
            solver="lbfgs",
            max_iter=5000,
            C=0.5,
            penalty="l2",
            n_jobs=-1,
            random_state=0
        )
    )

    clf.fit(train_features, train_labels)
    y_pred = clf.predict(test_features)
    score = clf.score(test_features, test_labels)
    tn, fp, fn, tp = confusion_matrix(test_labels, y_pred).ravel()
    return (fn, score)

In [16]:
def RandomForest(train_features, test_features, train_labels, test_labels):
  RandForest = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
  RandForest.fit(train_features, train_labels)
  forest_pred = RandForest.predict(test_features)
  score = RandForest.score(test_features,test_labels)
  tn, fp, fn, tp = confusion_matrix(test_labels, forest_pred).ravel()
  return(fn,score)


In [17]:
def KNN(train_features, test_features, train_labels, test_labels):
  #K Nearest Neighbor With Optimal Neighbor (1)
  knn = KNeighborsClassifier(n_neighbors = 1)
  knn.fit(train_features,train_labels)
  score = knn.score(test_features, test_labels)
  y_pred = knn.predict(test_features)
  tn, fp, fn, tp = confusion_matrix(test_labels, y_pred).ravel()
  #print(confusion_matrix(test_labels, y_pred))
  #print(classification_report(y_test, y_pred))
  return(fn,score)

In [18]:
def MLP(train_features, test_features, train_labels, test_labels):
  NeuralNetwork = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
  NeuralNetwork.fit(train_features, train_labels)
  nn_pred = NeuralNetwork.predict(test_features)
  score = NeuralNetwork.score(test_features,test_labels)
  #print(confusion_matrix(test_labels, nn_pred))
  tn, fp, fn, tp = confusion_matrix(test_labels, nn_pred).ravel()
  return(fn,score)

In [19]:
def SVM(train_features, test_features, train_labels, test_labels):
  clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
  clf.fit(train_features,train_labels)
  svmPred = clf.predict(test_features)
  score = clf.score(test_features,test_labels)
  #print(confusion_matrix(test_labels, pred))
  tn, fp, fn, tp = confusion_matrix(test_labels, svmPred).ravel()
  return(fn,score)

In [20]:
def tests(df, mDF):
  ogData = addLabel(makeSentence(df),0)
  testData = pd.concat([ogData,mDF])
  tokenized = testData[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
  max_len = 0
  for i in tokenized.values:
      if len(i) > max_len:
          max_len = len(i)

  padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

  train_features, test_features, train_labels, test_labels = train_test_split(padded, testData[1], random_state=42)
  lrFN,lrScore = logisticRegression(train_features, test_features, train_labels, test_labels)
  rfFN,rfScore = RandomForest(train_features, test_features, train_labels, test_labels)
  knnFN,knnScore = KNN(train_features, test_features, train_labels, test_labels)
  mlpFN,mlpScore = MLP(train_features, test_features, train_labels, test_labels)
  svmFN,svmScore = SVM(train_features, test_features, train_labels, test_labels)
  #print("test_labels: " + str(len(test_labels)))
  return(lrFN,lrScore,rfFN,rfScore,knnFN,knnScore,mlpFN,mlpScore,svmFN,svmScore)

## Processing

In [21]:
# df = pd.read_csv('https://raw.githubusercontent.com/HenryCWong/adversarialBERTMessages/master/weather_sample_data.csv', delimiter=',')
df = pd.read_csv('weather_sample_data.csv')

In [22]:
# df.drop("Unnamed: 0", axis=1, inplace=True) #unsafe copy
df = df.drop("Unnamed: 0", axis=1)

In [23]:
df.sample(5)

,temperature,outlook,wind,humidity
61,43,thunderstorm,NE,36
414,60,severe_thunderstorm,W,18
300,42,sunny,SE,70
272,45,sunny,W,88
174,50,rain,S,11


## Actual Stuffs

In [24]:
X, mDF = generator(df, 1)

In [25]:
display(mDF)

,0,1
0,"49,partly cloudy,NE,54",1
1,"44,sunny,S,51",1
2,"44,cloudy,SW,77",1
3,"55,cloudy,nan,29",1
4,"53,partly cloudy,SW,77",1
...,...,...
495,"55,partly cloudy,W,74",1
496,"49,thunderstorm,nan,19",1
497,"48,partly cloudy,NE,28",1
498,"55,rain,NW,42",1


In [26]:
tests(df, mDF) #logistic regression, random forest, knn, mlp, svm

/home/talos-1/miniconda3/envs/mitm_mqtt/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:602: ConvergenceWarning: lbfgs failed to converge after 13 iteration(s) (status=2):
ABNORMAL: 

You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


(np.int64(63),
 0.476,
 np.int64(23),
 0.624,
 np.int64(47),
 0.58,
 np.int64(88),
 0.468,
 np.int64(27),
 0.564)

In [27]:
score,gamma,indexes,jp,total = discriminator(X, 1, 1)

score: 0.696


In [28]:
print(jp)
print(total)

30
250


In [29]:
malDF = indexToEntry(X,indexes)

In [30]:
malDF.head()

,0,1
411,"50,thunderstorm,S,42",1
76,"43,severe_thunderstorm,NE,26",1
174,"50,severe_thunderstorm,S,40",1
370,"52,sunny,nan,52",1
59,"51,severe_thunderstorm,NW,52",1


In [ ]:
gamma = 1
rho = .1
sets = []
scores = []
fns = []
totals = []
gammas = []
lrFNs = []
rfFNs = []
knnFNs = []
mlpFNs = []
svmFNs = []
lrScores = []
rfScores = []
knnScores = []
mlpScores = []
svmScores = []
j = 0
while j < 30:
  X,mDF = generator(df, gamma)
  #print(gamma)
  score,gamma, indexes,fn,total = discriminator(X,gamma,rho)
  scores.append((j,score))
  #malDF = pd.concat([malDF,indexToEntry(X,indexes)])
  lrFN,lrScore,rfFN,rfScore,knnFN,knnScore,mlpFN,mlpScore,svmFN,svmScore = tests(df,mDF)
  #lists for stats
  lrFNs.append(lrFN)
  rfFNs.append(rfFN)
  knnFNs.append(knnFN)
  mlpFNs.append(mlpFN)
  svmFNs.append(svmFN)
  lrScores.append(lrScore)
  rfScores.append(rfScore)
  knnScores.append(knnScore)
  mlpScores.append(mlpScore)
  svmScores.append(svmScore)
  sets.append(indexToEntry(X,indexes))
  fns.append(fn)
  totals.append(total)
  gammas.append(gamma)
  j+=1


score: 0.724
score: 0.72
score: 0.68
score: 0.632


/home/talos-1/miniconda3/envs/mitm_mqtt/lib/python3.13/site-packages/sklearn/neural_network/_multilayer_perceptron.py:788: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


score: 0.568


In [ ]:
sortedScores = sorted(scores, key=lambda tup: tup[1])

In [ ]:
index,score = sortedScores[0]
malDF = sets[index]
for i in range(1,7):
  index, score = sortedScores[i]
  malDF = pd.concat([malDF,sets[index]])

In [ ]:
malDF

In [ ]:
df["temperature"].std()

In [ ]:
#malDF = pd.concat([malData,addLabel(makeSentence(df),0)])

## Graphing

In [ ]:
print(gammas)
print(fns)
print(totals)
print(scores)

In [ ]:
for a in scores:
  print(a[1])

In [ ]:
numScores = []
# [numScores.append(a[1]) for a in scores]
numScores = [s for (_, s) in scores]
print(numScores)

In [ ]:
sns.set_style("whitegrid")
sns.palplot(sns.color_palette("cubehelix", 5))
sns.set_palette("cubehelix",5)

In [ ]:
a4_dims = (15, 7)
fig, ax = plt.subplots(figsize=a4_dims)
ax.set(xlabel='Gamma', ylabel='Accuracy')
sns.lineplot(ax=ax,x=gammas,y=numScores)
#ax = sns.lineplot(x=gammas, y=numScores)


In [ ]:
a4_dims = (15, 7)
fig, ax = plt.subplots(figsize=a4_dims)
ax.set_xlabel('Gamma',fontsize=20);
ax.set_ylabel('Number of False Negatives',fontsize=20);
#ax.set(xlabel='Gamma', ylabel='False Positives')
sns.lineplot(ax=ax,x=gammas,y=fns,markers=True)
#ax = sns.lineplot(x=gammas, y=numScores)


In [ ]:
fpRatio = []
for i in range(len(fns)):
  fpRatio.append(fns[i]/totals[i])

In [ ]:
a4_dims = (15, 7)
fig, ax = plt.subplots(figsize=a4_dims)
ax.set(xlabel='Gamma', ylabel='False Negatives / Total Data Points')
sns.lineplot(ax=ax,x=gammas,y=fpRatio)


In [ ]:
d = {"gamma":gammas,"score":numScores,"fn":fns}
nDF = pd.DataFrame(data=d)

In [ ]:
a4_dims = (15, 7)

fig = plt.figure(figsize=a4_dims)
a1 = fig.add_axes([0,0,1,1])
a1.plot(nDF["gamma"],nDF["score"],color="orchid",marker="x",linewidth=10.0)
a1.set_ylabel('Accuracy',fontsize = 30)
a1.set_xlabel("Gamma",fontsize = 30)
a2 = a1.twinx()
a2.plot(nDF["gamma"],nDF["fn"],marker="s",linewidth=10.0)
a2.set_ylabel('False Negatives',fontsize = 30)
a2.set_xlabel("Gamma",fontsize = 30)
fig.legend(labels = ('Accuracy','False Negatives'),loc='upper center',prop={'size': 30})
plt.show()

In [ ]:
#Iterpolation attempt
from scipy.interpolate import interp1d

In [ ]:
f1 = interp1d(nDF["gamma"], nDF['score'],kind='cubic')
f2 = interp1d(nDF["gamma"], nDF['fn'],kind='cubic')
xnew = np.arange(1.1,4,.2)
df2 = pd.DataFrame()
df2['score'] = f1(xnew)
df2['fn'] = f2(xnew)
df2["xnew"] = xnew
a4_dims = (15, 7)

fig = plt.figure(figsize=a4_dims)
a1 = fig.add_axes([0,0,1,1])
a1.plot(df2["xnew"],df2["score"],color="red",linewidth=2.0)
a1.set_ylabel('Accuracy',fontsize = 30)
a1.set_xlabel("Gamma",fontsize = 30)
a2 = a1.twinx()
a2.plot(df2["xnew"],df2["fn"],linewidth=2.0)
a2.set_ylabel('False Negatives',fontsize = 30)
a2.set_xlabel("Gamma",fontsize = 30)
fig.legend(labels = ('Accuracy','False Negatives'),prop={'size': 20})
plt.show()

In [ ]:
import scipy

x_new = np.linspace(1, 4, 300)
# a_BSpline = scipy.interpolate.make_interp_spline(x, y)
# y_new = a_BSpline(x_new)

In [ ]:
f1 = interp1d(nDF["gamma"], nDF['score'],kind='cubic')
f2 = interp1d(nDF["gamma"], nDF['fn'],kind='cubic')

xnew = np.arange(1.1,4,.3)
df2 = pd.DataFrame()
df2['Weight_A'] = f1(xnew)
df2['Weight_B'] = f2(xnew)
print(df2)
ax2 = df2.plot.line()
ax2.set_title('After interpolation')
ax2.set_xlabel("year") #???
ax2.set_ylabel("weight")


plt.show()

In [ ]:
d = {"Logistic Regression":lrFNs,"Random Forest":rfFNs,"K Nearest Neighbor":knnFNs,"Multi-Layer Perceptron":mlpFNs,"Support Vector Classification":svmFNs,"gamma":gammas}
#n = np.array([lrFNs,rfFNs,knnFNs,mlpFNs,svmFNs])
mlstuffs = pd.DataFrame(d)

In [ ]:
print(svmScores)

In [ ]:
s = {"Logistic Regression":lrScores,"Random Forest":rfScores,"K Nearest Neighbor":knnScores,"Multi-Layer Perceptron":mlpScores,"Support Vector Classification":svmScores,"gamma":gammas}
scoreStuffs = pd.DataFrame(s)

In [ ]:
meltedScores = scoreStuffs.melt(id_vars=['gamma'])
meltedScores = scoreStuffs.melt(id_vars=['gamma'])
meltedScores.columns = ["gamma","Classification Model","Accuracy"]

In [ ]:
meltedML = mlstuffs.melt(id_vars=['gamma'])
meltedML.columns = ["gamma","Classification Model","Number of False Negatives"]

In [ ]:
# from google.colab import drive
# drive.mount('drive')

In [ ]:
meltedScores.to_csv('meltedScores.csv')
meltedML.to_csv("meltedML.csv")
# !cp meltedScores.csv "drive/My Drive/Research"
# !cp meltedML.csv "drive/My Drive/Research"

In [ ]:
meltedML.head()

In [ ]:
a4_dims = (15, 10)
fig, ax = plt.subplots(figsize=a4_dims)
#ax.set(xlabel='Gamma', ylabel='False Negatives')
ax.set_xlabel('Gamma',fontsize=20);
ax.set_ylabel('Number of False Negatives',fontsize=20);
sns.lineplot(ax=ax,x="gamma",y="Number of False Negatives",hue="Classification Model",data=meltedML)
#this displays during trainig and testing "how many malicious messages made it through" at different gammas

In [ ]:
#plt.plot(meltedML[meltedML["Classification Model"] == "Logistic Regression"])
a4_dims = (15, 10)
fig, ax = plt.subplots(figsize=a4_dims)
#ax.set(xlabel='Gamma', ylabel='False Negatives')
ax.set_xlabel('Gamma',fontsize=30);
ax.set_ylabel('Number of False Negatives',fontsize=30);
models = ["Logistic Regression","Random Forest","K Nearest Neighbor","Multi-Layer Perceptron","Support Vector Classification"]
markers = ["x","^","*","x","s"]
for i in range(len(models)):
  a = meltedML[meltedML["Classification Model"] == models[i]]
  if models[i] == "Multi-Layer Perceptron" or models[i] == "K Nearest Neighbor":
    plt.plot(a["gamma"],a["Number of False Negatives"],linewidth=10.0,marker=markers[i])
  else:
    plt.plot(a["gamma"],a["Number of False Negatives"],linewidth=1.0,marker=markers[i])
plt.legend(models,prop={'size': 20})

#lr = meltedML[meltedML["Classification Model"] == "Logistic Regression"]
#rf = meltedML[meltedML["Classification Model"] == "Random Forest"]

#plt.plot(lr["gamma"],lr["Number of False Negatives"])
#plt.plot(rf["gamma"],rf["Number of False Negatives"])


In [ ]:
meltedScores.head()

In [ ]:
a4_dims = (15, 7)
fig, ax = plt.subplots(figsize=a4_dims)
#ax.set(xlabel='Gamma', ylabel='False Negatives')
ax.set_xlabel('Gamma',fontsize=20);
ax.set_ylabel('Accuracy',fontsize=20);
sns.lineplot(ax=ax,x="gamma",y="Accuracy",hue="Classification Model",data=meltedScores)
#this displays during trainig and testing "how many malicious messages made it through" at different gammas

In [ ]:
a4_dims = (15, 10)
fig, ax = plt.subplots(figsize=a4_dims)
ax.set_xlabel('Gamma',fontsize=30);
ax.set_ylabel('Accuracy',fontsize=30);
models = ["Logistic Regression","Random Forest","K Nearest Neighbor","Multi-Layer Perceptron","Support Vector Classification"]
markers = ["x","^","*","x","s"]
for i in range(len(models)):
  a = meltedScores[meltedScores["Classification Model"] == models[i]]
  if models[i] == "Multi-Layer Perceptron" or models[i] == "Support Vector Classification":
    plt.plot(a["gamma"],a["Accuracy"],linewidth=10.0,marker=markers[i])
  else:
    plt.plot(a["gamma"],a["Accuracy"],linewidth=1.0,marker=markers[i])
plt.legend(models,prop={'size': 20})

#lr = meltedML[meltedML["Classification Model"] == "Logistic Regression"]
#rf = meltedML[meltedML["Classification Model"] == "Random Forest"]

#plt.plot(lr["gamma"],lr["Number of False Negatives"])
#plt.plot(rf["gamma"],rf["Number of False Negatives"])


# Attempted Test

In [ ]:
ogData = addLabel(makeSentence(df),0)

In [ ]:
testData = pd.concat([ogData,malDF])

In [ ]:
testData

In [ ]:
testData[testData[1] == 1]

In [ ]:
tokenized = testData[0].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [ ]:
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

In [ ]:
padded

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(padded, testData[1], random_state=42)

In [ ]:
test_labels.value_counts()

### Logistic Regression Test

In [ ]:
lr_clf = LogisticRegression(max_iter=500)
lr_clf.fit(train_features, train_labels)
print(lr_clf.score(test_features, test_labels))
y_pred = lr_clf.predict(test_features)

In [ ]:
from sklearn.metrics import classification_report

# confusion_matrix(test_labels, y_pred)
print(classification_report(test_labels, y_pred))

### Random Forest Test

In [ ]:
RandForest = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
RandForest.fit(train_features, train_labels)
forest_pred = RandForest.predict(test_features)
print(RandForest.score(test_features,test_labels))

In [ ]:
confusion_matrix(test_labels, forest_pred)

### KNN

In [ ]:
correct = []
for i in range(1, 20):
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(train_features,train_labels)
    prediction = knn.predict(test_features)
    correct.append(np.mean(prediction == test_labels))

plt.figure(figsize=(10, 5))
plt.plot(range(1, 20), correct)
plt.title('% Correct vs. K Value')
plt.xlabel('K Value')
plt.ylabel('% Correct')


#K Nearest Neighbor With Optimal Neighbor (1)
knn = KNeighborsClassifier(n_neighbors = 1)
knn.fit(train_features,train_labels)
print("Accuracy " + str(knn.score(test_features, test_labels)))
y_pred = knn.predict(test_features)
print(confusion_matrix(test_labels, y_pred))
#print(classification_report(y_test, y_pred))

### Neural Networks

In [ ]:
NeuralNetwork = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
NeuralNetwork.fit(train_features, train_labels)
nn_pred = NeuralNetwork.predict(test_features)
print(NeuralNetwork.score(test_features,test_labels))
print(confusion_matrix(test_labels, nn_pred))

### SVC

In [ ]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(train_features,train_labels)

In [ ]:
pred = clf.predict(test_features)

In [ ]:
# print(confusion_matrix(test_labels, pred))
print(classification_report(test_labels, pred))

### OCSVM